In [2]:
import xml.etree.ElementTree as ET

# Function returns a "tracks" and metadata variables:
    # "tracks" is a set of nested lists for each set of coords in each trajectory like so:
        # [[[1,x,y,z],[2,x,y,z]], [traj2], [traj3], etc.]
        # timepoint/frame, x-coord, y-coord, 0.0 (2D trajectories)
def import_trackmate_tracks(file, clipz=False, scalet=False):
    try:
        doc = ET.parse(file)
    except ET.ParseError:
        raise ValueError(f'Failed to read XML file {file}')
    
    root = doc.getroot()
    
    if root.tag != 'Tracks':
        raise ValueError('File does not seem to be a proper track file.')
    
    metadata = {
        # Pixels
        'spaceUnits': root.get('spaceUnits'),
        # Frames
        'timeUnits': root.get('timeUnits'),
        # milliseconds
        'frameInterval': float(root.get('frameInterval')),
        # Date
        'date': root.get('generationDateTime'),
        # TrackMate software
        'source': root.get('from')
    }
    
    tracks = []
    track_nodes = root.findall('particle')
    
    for track_node in track_nodes:
        n_spots = int(track_node.get('nSpots'))
        track = []
        detection_nodes = track_node.findall('detection')
        for detection_node in detection_nodes:
            t = float(detection_node.get('t'))
            x = float(detection_node.get('x'))
            y = float(detection_node.get('y'))
            z = float(detection_node.get('z'))
            track.append([t, x, y, z])
        tracks.append(track)
    
    if clipz:
        if all(all(point[3] == 0 for point in track) for track in tracks):
            for i in range(len(tracks)):
                tracks[i] = [point[:3] for point in tracks[i]]
    
    if scalet and not any(track[0][0] % 1 != 0 for track in tracks):
        frame_interval = metadata['frameInterval']
        for i in range(len(tracks)):
            tracks[i] = [[point[0] * frame_interval] + point[1:] for point in tracks[i]]
    
    return tracks, metadata